In [1]:
import torch
from transformers import AutoTokenizer, Trainer, TrainingArguments, AutoModelForSequenceClassification
from torch.utils.data import Dataset
import os
import pandas as pd
import requests
from tqdm.auto import tqdm
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, matthews_corrcoef
import re
import warnings

warnings.filterwarnings("ignore")

In [2]:
model_name = './prot_bert_bfd'

In [3]:
class DeepLocDataset(Dataset):
    def __init__(self, split="train", tokenizer_name='Rostlab/prot_bert', max_length=1024):
        self.datasetFolderPath = 'dataset/'
        self.trainFilePath = os.path.join(self.datasetFolderPath, 'train.csv')
        self.testFilePath = os.path.join(self.datasetFolderPath, 'test.csv')
        self.tokenizer = AutoTokenizer.from_pretrained(tokenizer_name, do_lower_case=False)
        if split=="train":
            self.seqs, self.labels = self.load_dataset(self.trainFilePath)
        else:
            self.seqs, self.labels = self.load_dataset(self.testFilePath)
        self.max_length = max_length
        
    def load_dataset(self,path):
        df = pd.read_csv(path,names=['input','labels'],skiprows=1)
        seq = list(df['input'])
        seq = [' '.join(i) for i in seq]
        label = list(df['labels'])
        assert len(seq) == len(label)
        return seq, label

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        seq = " ".join("".join(self.seqs[idx].split()))
        seq = re.sub(r"[UZOB]", "X", seq)
        seq_ids = self.tokenizer(seq, truncation=True, padding='max_length', max_length=self.max_length)
        sample = {key: torch.tensor(val) for key, val in seq_ids.items()}
        sample['labels'] = torch.tensor(self.labels[idx])
        return sample

In [4]:
train_dataset = DeepLocDataset(split="train", tokenizer_name=model_name, max_length=80)
test_dataset = DeepLocDataset(split="test", tokenizer_name=model_name, max_length=80)

In [5]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall,
        'mcc' : matthews_corrcoef(labels, preds)
    }

In [6]:
def model_init():
      return AutoModelForSequenceClassification.from_pretrained(model_name)#.cuda()

In [7]:
training_args = TrainingArguments(
    learning_rate=1e-4,
    output_dir='./results',
    num_train_epochs=26,
    per_device_train_batch_size=10,
    per_device_eval_batch_size=10,
    warmup_steps=10,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=2,
    do_train=True,
    do_eval=True,
    evaluation_strategy="epoch",
    gradient_accumulation_steps=64,
    fp16=True,
    fp16_opt_level="02",
    run_name="ProBert-BFD-MS",
    seed=3407
)

# 26

trainer = Trainer(
    model_init=model_init,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics = compute_metrics,
)

trainer.train()

Some weights of the model checkpoint at ./prot_bert_bfd were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialize

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,Mcc,Runtime,Samples Per Second
0,0.693500,0.690898,0.543478,0.704225,0.543478,1.000000,0.000000,2.375700,193.629000
1,0.995700,0.683549,0.845652,0.870201,0.801347,0.952000,0.698813,2.351900,195.584000
2,0.977200,0.643123,0.876087,0.888889,0.866920,0.912000,0.750273,2.415400,190.444000
3,0.912200,0.578115,0.884783,0.891616,0.912134,0.872000,0.769665,2.493200,184.499000
4,0.800800,0.520068,0.904348,0.907563,0.955752,0.864000,0.813412,2.305500,199.520000
5,0.683900,0.422432,0.878261,0.879310,0.953271,0.816000,0.767329,2.364400,194.551000
6,0.562000,0.330990,0.913043,0.917695,0.944915,0.892000,0.827232,2.347700,195.937000
7,0.434900,0.307806,0.904348,0.912698,0.905512,0.920000,0.807068,2.312000,198.960000
8,0.366900,0.273028,0.921739,0.925926,0.953390,0.900000,0.844696,2.428300,189.436000
9,0.320000,0.307036,0.902174,0.906445,0.943723,0.872000,0.807034,2.188400,210.197000


TrainOutput(global_step=52, training_loss=0.3542031609954742, metrics={'train_runtime': 845.6425, 'train_samples_per_second': 0.061, 'total_flos': 9520053299894400, 'epoch': 25.7})

In [8]:
trainer.evaluate(test_dataset)

{'eval_loss': 0.26451951265335083,
 'eval_accuracy': 0.9347826086956522,
 'eval_f1': 0.9397590361445783,
 'eval_precision': 0.9435483870967742,
 'eval_recall': 0.936,
 'eval_mcc': 0.8687047839301032,
 'eval_runtime': 2.203,
 'eval_samples_per_second': 208.803,
 'epoch': 25.7}

In [9]:
trainer.save_model('./model/bertbfd937.pt')

In [10]:
result = trainer.predict(test_dataset).predictions
golden = trainer.predict(test_dataset).label_ids
result = torch.argmax(torch.tensor(result), -1)

In [11]:
result

tensor([1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0,
        1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1,
        0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1,
        1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1,
        1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0,
        1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1,
        1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0,
        1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1,
        0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0,
        1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0,
        1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0,
        1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0,